In [ ]:
def butter_filter(data_frame,sensor,cutoff,order,sampling_rate,unit,component_name):
    
    sos = signal.butter(order, cutoff, 'lp', fs=sampling_rate, output='sos')
    filt_sensor = signal.sosfilt(sos, sensor)
    sample_spacing = 1/sampling_rate

    filt_sensor_FT=fft.fft(filt_sensor)
    filt_sensor_FT_freq=fft.fftfreq(filt_sensor.size,sample_spacing)
    N = int(filt_sensor_FT_freq.size/2) 
    filt_sensor_freq_pos = filt_sensor_FT_freq[:N]
    filt_sensor_FT_pos = np.abs(filt_sensor_FT[:N])
    
    with plt.rc_context({"figure.figsize" : (20,3)}):
        fig,axs = plt.subplots(1,2)
        axs[0].plot(data_frame.index,filt_sensor)
        axs[0].set_title(component_name)
        axs[0].set_xlabel('time [s]')
        axs[0].set_ylabel(unit)
        axs[1].plot(filt_sensor_FT_freq[:N],np.abs(filt_sensor_FT[:N]))
        axs[1].set_title("FT")
        axs[1].set_xlabel('frequency [Hz]')
        axs[1].set_ylabel('power');
        
    def f(x,a,b,c,d):
        return a*np.exp(-pow(x-b,2)/(2*pow(c,2)))+d 
    
    x=filt_sensor_FT_freq[:N]
    y=np.abs(filt_sensor_FT[:N])

    zoom=np.where(x<cutoff+0.5)
    power_trashold=200

    peak=np.where(y==y.max())
    peaks=np.array(x[np.where(y>=power_trashold)])
    power_peaks=np.array(y[np.where(y>=power_trashold)])
    
    
    mask=np.where(abs(x-x[peak])<=1/np.sqrt(12)) ### da definire meglio
    

    popt, pcov = curve_fit(f, x[mask], y[mask]) 
    x_range=np.linspace(x[mask].min(),x[mask].max(),1000) 

    fig,ax = plt.subplots(1,2,figsize=(20,4))
    ax[0].plot(x[zoom],y[zoom])
    #ax[0].scatter(peaks,power_peaks,marker='.',color='red')
    ax[0].set_title('FT')
    ax[0].set_xlabel('frequency [Hz]')
    ax[0].set_ylabel('power')
    ax[1].plot(x[mask],y[mask])
    ax[1].plot(x_range, f(x_range,*popt), c='red', linestyle='dashed' ,label='Gaussian fit',)
    ax[1].legend(loc='best')
    ax[1].set_title('FT')
    ax[1].set_xlabel('frequency [Hz]')
    ax[1].set_ylabel('power');

    err_perc=abs(popt[2]*100/popt[1])
    RR=60*popt[1]
    RR_error=popt[2]*60

    print(component_name)
    print('# of points to perform fit:',len(x[mask]))
    print('Fit parametres for RR:\n', popt, '\n')
    print('Respiratory rate estimated: {:.2} +/- {:.2}'.format(RR, RR_error))
    print('Percentage of error: {:.2} % \n'.format(err_perc))
    
    return RR, RR_error

In [ ]:
butter_filter(df,magnT[:,1],0.5,10,sampling_rate,'[G]','Second magnetoscope PCA component filtred')

In [ ]:
##### secondo dataset

In [ ]:
selected_PCA_df2=[accT2[:,0],gyroT2[:,0],magnT2[:,0]]
units=['[mg]','[DPS]','[G]']
component_name=['First accelerometer PCA component filtred', 'First gyroscope PCA component filtred','First magnetoscope PCA component filtred']
RR_butter1=np.zeros(len(selected_PCA_df2))
RR_butter_error1=np.zeros(len(selected_PCA_df2))

for id,(i,j,k) in enumerate(zip(selected_PCA_df2,units,component_name)):   
    RR_butter1[id],RR_butter_error1[id]=butter_filter(df2,i,0.5,10,sampling_rate,j,k) 
    

In [ ]:
av_butter,av_butter_error=np.average(RR_butter,weights=1/RR_butter_error,returned=True)
av_butter_error_perc=abs((av_butter_error*100)/av_butter)

print('RR estimeted via weighted average: {:.2} +/- {:.2}'.format(av_butter,av_butter_error))
print('Percentage of error: {:.2} % \n'.format(av_butter_error_perc))